Teraz, mając rozkład $M^TM = \bar{U} \Sigma \bar{U}^{-1}$, trzeba się zastanowić co dalej -- jak zrobić z tego SVD macierzy $M$?

Niech $M = X\Lambda Y^T$ będzie szukanym SVD. Wiemy, że:
1. $\Lambda$ jest diagonalna i na diagonali ma pierwiastki z elementów diagonali $\Sigma$.
2. kolumny $Y$ są wektorami własnymi $M^TM$, czyli $Y = U$. Jak teraz wyznaczyć $X$?
3. skoro znamy $M$, to mając równość $M = X\Lambda Y^T$ mamy $X = (\Lambda Y^T)^{-1}M = Y^{-T}\Lambda^{-1}M = Y\Lambda^{-1} M$.

Wówczas, skoro $M = P_m C P_m^T$, to $C = P_m ^T M P_m = P_m ^T X \Lambda Y P_m$, czyli:
$$B = \begin{bmatrix}
U_1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & U_2
\end{bmatrix}P_m ^T X \Lambda Y P_m\begin{bmatrix}
V_1 ^T & 0\\
0 & V_2 ^T
\end{bmatrix}$$